**Blok 1: Import Library**

In [1]:
# Basic libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import time
import copy
from collections import Counter

# PyTorch libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms, models, datasets
from torchvision.transforms import v2

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")

All libraries imported successfully!


**Blok 2: Cek GPU dan Setup Device**

In [2]:
# Check GPU availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"Number of GPUs: {torch.cuda.device_count()}")

if torch.cuda.is_available():
  current_device = torch.cuda.current_device()
  gpu_properties = torch.cuda.get_device_properties(current_device)
  print(f"GPU Name: {gpu_properties.name}")
  print(f"GPU Memory: {gpu_properties.total_memory / 1024**3:.2f} GB")
  print(f"GPU Compute Capability: {gpu_properties.major}.{gpu_properties.minor}")
else:
  print("No GPU available, using CPU")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nUsing device: {device}")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
  torch.cuda.manual_seed(42)

PyTorch version: 2.9.0+cu130
CUDA available: True
CUDA version: 13.0
Number of GPUs: 1
GPU Name: NVIDIA GeForce RTX 4060 Laptop GPU
GPU Memory: 8.00 GB
GPU Compute Capability: 8.9

Using device: cuda


**Blok 3: Setup Path dan Parameters**

In [3]:
# Define dataset paths
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Check if directories exist
print("Checking dataset structure...")
if os.path.exists(train_dir) and os.path.exists(test_dir):
  train_classes = os.listdir(train_dir)
  test_classes = os.listdir(test_dir)
  print(f"Train classes: {train_classes}")
  print(f"Test classes: {test_classes}")
    
  # Count number of images per class
  print("\nNumber of images per class in train set:")
  for class_name in train_classes:
    class_path = os.path.join(train_dir, class_name)
    if os.path.isdir(class_path):
      num_images = len(os.listdir(class_path))
      print(f"  {class_name}: {num_images} images")

  print("\nNumber of images per class in test set:")
  for class_name in test_classes:
    class_path = os.path.join(test_dir, class_name)
    if os.path.isdir(class_path):
      num_images = len(os.listdir(class_path))
      print(f"  {class_name}: {num_images} images")
else:
  print(f"Error: Dataset directories not found!")
  print(f"Train directory: {train_dir}")
  print(f"Test directory: {test_dir}")

# Training parameters
batch_size = 64
num_epochs = 50
num_classes = 7

Checking dataset structure...
Train classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Test classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

Number of images per class in train set:
  angry: 3995 images
  disgust: 436 images
  fear: 4097 images
  happy: 7215 images
  neutral: 4965 images
  sad: 4830 images
  surprise: 3171 images

Number of images per class in test set:
  angry: 958 images
  disgust: 111 images
  fear: 1024 images
  happy: 1774 images
  neutral: 1233 images
  sad: 1247 images
  surprise: 831 images


**Blok 4: Data Transforms dan Augmentation**

In [4]:
# Enhanced data transformations with heavy augmentation
train_transform = v2.Compose([
  v2.Resize((224, 224)),  # Resize for ResNet
  v2.RandomHorizontalFlip(p=0.5),
  v2.RandomRotation(degrees=15),
  v2.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
  v2.ColorJitter(brightness=0.2, contrast=0.2),
  v2.RandomGrayscale(p=0.1),
  v2.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
  v2.ToImage(),
  v2.ToDtype(torch.float32, scale=True),
  v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

val_transform = v2.Compose([
  v2.Resize((224, 224)),
  v2.ToImage(),
  v2.ToDtype(torch.float32, scale=True),
  v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = val_transform  # Same as validation transform

print("Data transforms defined successfully!")

Data transforms defined successfully!


**Blok 5: Load Dataset dengan ImageFolder**

In [ ]:
# Load datasets using ImageFolder
print("Loading datasets...")

try:
  train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
  test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)
    
  # Get class names
  class_names = train_dataset.classes
  print(f"Class names: {class_names}")
  print(f"Number of classes: {len(class_names)}")

  print(f"Training samples: {len(train_dataset)}")
  print(f"Test samples: {len(test_dataset)}")
    
  # Split train into train and validation (90% train, 10% validation)
  train_size = int(0.9 * len(train_dataset))
  val_size = len(train_dataset) - train_size

  train_dataset, val_dataset = torch.utils.data.random_split(
    train_dataset, [train_size, val_size]
  )
    
  # Apply validation transform to validation set
  val_dataset.dataset = copy.deepcopy(val_dataset.dataset)
  val_dataset.dataset.transform = val_transform

  print(f"After split - Training samples: {len(train_dataset)}")
  print(f"After split - Validation samples: {len(val_dataset)}")
  print(f"After split - Test samples: {len(test_dataset)}")

except Exception as e:
  print(f"Error loading datasets: {e}")
  # Create dummy datasets for demonstration if real datasets not available
  from torchvision.datasets import FakeData

  train_dataset = FakeData(size=28000, image_size=(3, 48, 48), num_classes=7, 
                        transform=train_transform)
  val_dataset = FakeData(size=3500, image_size=(3, 48, 48), num_classes=7, 
                        transform=val_transform)
  test_dataset = FakeData(size=3500, image_size=(3, 48, 48), num_classes=7, 
                        transform=test_transform)
  class_names = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']